# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [3]:
import pandas as pd
from gensim import corpora

# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])

pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds))

{'game': 28115,
 'german': 110,
 'involve': 286,
 'long': 1689,
 'politic': 24,
 'political': 70,
 'hard': 1027,
 'intrigue': 39,
 'love': 2026,
 'play': 13867,
 'player': 6769,
 'try': 1797,
 'drawback': 37,
 'incredible': 72,
 'length': 266,
 'rule': 2633,
 'translation': 26,
 'cardboard': 96,
 'excellent': 604,
 'nice': 1704,
 'piece': 525,
 'token': 352,
 'balanced': 254,
 'fairly': 375,
 'sister': 22,
 'tend': 158,
 'collection': 658,
 'core': 256,
 'mid': 99,
 'strategy': 1621,
 'elegant': 159,
 'highly': 276,
 'score': 820,
 'simple': 1577,
 'abstract': 469,
 'board': 1902,
 'charm': 42,
 'island': 84,
 'japanese': 34,
 'paste': 54,
 'shape': 77,
 'sort': 267,
 'theme': 2257,
 'true': 152,
 'work': 853,
 'badly': 54,
 'draw': 749,
 'like': 6490,
 'screw': 115,
 'think': 2338,
 'tile': 1237,
 'big': 978,
 'fan': 526,
 'luck': 1294,
 'ruin': 86,
 'sale': 84,
 'trade': 348,
 'available': 308,
 'bean': 23,
 'card': 5601,
 'edition': 649,
 'miss': 358,
 'name': 60,
 'number': 615,
 '

In [9]:
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds))

game         0.411098
german       0.001608
involve      0.004182
long         0.024697
politic      0.000351
               ...   
muppet       0.000015
pigs         0.000015
coerce       0.000015
hibernate    0.000015
improper     0.000015
Name: 0, Length: 19749, dtype: float64

Ar 'game'(40%), 'play'(20%) and '<game_name>'(11%) stopwords? <br>
For sure 'game' is a stopword, what for the other two? I believe they bring no context.
We remove them!

In [1]:
stop_words = ['game', 'play', '<game_name>']

In [2]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
default_config = LdaGeneratorConfig(corpus_file_path=corpus_path)
lda, dictionary = LdaModelGenerator(default_config, stop_words).make_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

In [3]:
from gensim.models import CoherenceModel
import pandas as pd

test_ds = pd.read_csv("../dataset/output/default_sentences/pre_processed.80k.test.csv")

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
coh = CoherenceModel(lda, texts=texts, coherence='c_v', topn=topn)
print(f"Model perplexity: {lda.log_perplexity(texts.apply(lambda x: dictionary.doc2bow(x)).tolist())}")
print(f"On topn={topn} we have:")
print(f"Model coherence of: {coh.get_coherence()}")
print("Coherence per topic:")
coh.get_coherence_per_topic()

Model perplexity: -28.539873341336463
On topn=10 we have:
Model coherence of: 0.3806187801058944
Coherence per topic:


[0.30619673418016713,
 0.625321044315345,
 0.3451918285525357,
 0.3961520295908048,
 0.3716536346661269,
 0.31241651938227855,
 0.38341617253487703,
 0.37256915384219746,
 0.3643398730431328,
 0.462646736713212,
 0.37025132057848176,
 0.24999385809652908,
 0.4016976786890141,
 0.3668163372978198]

In [4]:
lda.show_topics(20, 20)

[(0,
  '0.056*"know" + 0.049*"use" + 0.038*"thing" + 0.030*"similar" + 0.029*"actually" + 0.025*"option" + 0.021*"worth" + 0.021*"series" + 0.020*"good" + 0.018*"player" + 0.017*"copy" + 0.017*"die" + 0.017*"see" + 0.016*"sell" + 0.015*"update" + 0.015*"fact" + 0.015*"word" + 0.014*"plus" + 0.014*"basically" + 0.014*"available"'),
 (1,
  '0.234*"card" + 0.057*"board" + 0.039*"box" + 0.029*"big" + 0.028*"draw" + 0.025*"hand" + 0.025*"hard" + 0.024*"small" + 0.019*"ability" + 0.018*"player" + 0.018*"piece" + 0.017*"pick" + 0.017*"setup" + 0.017*"need" + 0.015*"story" + 0.015*"action" + 0.015*"track" + 0.014*"opponent" + 0.013*"special" + 0.012*"run"'),
 (2,
  '0.082*"lot" + 0.065*"long" + 0.061*"new" + 0.052*"take" + 0.035*"group" + 0.034*"design" + 0.032*"time" + 0.028*"player" + 0.021*"good" + 0.020*"fun" + 0.019*"rule" + 0.018*"write" + 0.017*"say" + 0.016*"problem" + 0.016*"leave" + 0.016*"deep" + 0.015*"bit" + 0.014*"campaign" + 0.013*"expect" + 0.012*"trick"'),
 (3,
  '0.083*"point

### POS: Nouns only

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

In [29]:
# Now we try to work on noun only to see if we get a good representation.
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.csv"
pos_tagged_ds = pd.read_csv(corpus_path)['comments']

ds = pos_tagged_ds.swifter.apply(lambda x: x.split(' '))
ds = ds.swifter.apply(lambda x: [w.split('__noun')[0] for w in x if w.endswith('__noun')])
ds = ds[ds.map(len) > 1].map(lambda x: ' '.join(x)).drop_duplicates()

corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
ds.to_csv(corpus_path, index=False)

# Now we also make the test set
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.test.csv"
pos_tagged_ds = pd.read_csv(test_corpus_path)['comments']

ds = pos_tagged_ds.swifter.apply(lambda x: x.split(' '))
ds = ds.swifter.apply(lambda x: [w.split('__noun')[0] for w in x if w.endswith('__noun')])
ds = ds[ds.map(len) > 1].map(lambda x: ' '.join(x)).drop_duplicates()

test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"
ds.to_csv(test_corpus_path, index=False)

Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/60701 [00:00<?, ?it/s]

In [9]:
import pandas as pd
from gensim import corpora

# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

count_dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(count_dictionary)):
    doc_frequency[count_dictionary.get(index)] = len(ds[[count_dictionary.get(index) in x for x in ds]])

pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds))

drawback       0.001420
game           0.683538
length         0.008883
translation    0.001170
piece          0.018054
token          0.011013
fun            0.116711
sister         0.000940
board          0.061569
charm          0.001497
Name: 0, dtype: float64

In [6]:
# Default approach on the sentences only to avoid global topic recognition
from model import LdaModelGenerator, LdaGeneratorConfig

stop_words = ['game', 'play', '<game_name>']
corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.csv"
default_config = LdaGeneratorConfig(corpus_file_path=corpus_path)
noun_lda, noun_dictionary = LdaModelGenerator(default_config, stop_words).make_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/52120 [00:00<?, ?it/s]

In [7]:
from gensim.models import CoherenceModel
import pandas as pd

test_corpus_path = "../dataset/output/pos_tagged/pre_processed.80k.noun_only.test.csv"
test_ds = pd.read_csv(test_corpus_path)

# For the c_v model
texts = test_ds['comments'].apply(lambda x: x.split(' '))

topn = 10  # For the coherence evaluation.
coh = CoherenceModel(noun_lda, texts=texts, coherence='c_v', topn=topn)
print(f"Model perplexity: {noun_lda.log_perplexity(texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist())}")
print(f"On topn={topn} we have:")
print(f"Model coherence of: {coh.get_coherence()}")
print("Coherence per topic:")
coh.get_coherence_per_topic()

Model perplexity: -20.254782829421483
On topn=10 we have:
Model coherence of: 0.521321437417917
Coherence per topic:


[0.41723452087712937,
 0.8103165457515475,
 0.6801578855107604,
 0.3912342857105625,
 0.5440103502257652,
 0.39467163953274637,
 0.5219267761869469,
 0.4897019448296467,
 0.6979860354360248,
 0.4267472846778877,
 0.4671028205934368,
 0.5115350579793349,
 0.49584457938416193,
 0.45003039715488774]

In [49]:
noun_lda.show_topics(20, 20)

[(0,
  '0.170*"game" + 0.144*"rule" + 0.102*"play" + 0.059*"rating" + 0.030*"lot" + 0.020*"mode" + 0.019*"system" + 0.016*"brain" + 0.016*"battle" + 0.016*"series" + 0.015*"head" + 0.015*"designer" + 0.014*"enemy" + 0.013*"book" + 0.012*"depth" + 0.012*"rulebook" + 0.011*"wargame" + 0.011*"way" + 0.011*"house" + 0.011*"company"'),
 (1,
  '0.148*"game" + 0.143*"<game_name>" + 0.104*"version" + 0.097*"worker" + 0.095*"placement" + 0.074*"strategy" + 0.062*"art" + 0.027*"euro" + 0.024*"work" + 0.022*"idea" + 0.022*"taste" + 0.018*"aspect" + 0.016*"randomness" + 0.015*"half" + 0.014*"memory" + 0.014*"reason" + 0.011*"racing" + 0.011*"mechanism" + 0.010*"fight" + 0.008*"chess"'),
 (2,
  '0.074*"game" + 0.071*"collection" + 0.057*"map" + 0.048*"space" + 0.047*"board" + 0.038*"bonus" + 0.034*"weight" + 0.033*"drafting" + 0.028*"price" + 0.024*"addition" + 0.023*"word" + 0.021*"world" + 0.021*"trade" + 0.020*"improvement" + 0.020*"train" + 0.019*"wife" + 0.017*"text" + 0.017*"change" + 0.016*"

In [ ]:
# Now the model has high coherence which is good but are the extracted aspects good?
# Probably not.

In [5]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.125996  0.145912       1        1  10.523325
4      0.022883 -0.115903       2        1   8.360380
1      0.200001 -0.105780       3        1   8.134769
3      0.064468  0.057080       4        1   7.454352
6     -0.169962 -0.080337       5        1   7.244243
7     -0.135145  0.182267       6        1   7.100618
2     -0.159558  0.005533       7        1   7.039122
12     0.071411  0.144753       8        1   7.034553
11    -0.119070 -0.037482       9        1   6.900046
5      0.005623  0.118006      10        1   6.752671
13    -0.100775 -0.159993      11        1   6.316563
9      0.187775 -0.213361      12        1   5.815936
0     -0.011665 -0.152084      13        1   5.683028
8      0.270010  0.211389      14        1   5.640394, topic_info=            Term         Freq        Total Category  logprob  loglift
57          card  2926.000000  2926.000000  Default  30.0000  30.0000
45          like  2676.000000  2676.000000  Default  29.0000  29.0000
85          time  1659.000000  1659.000000  Default  28.0000  28.0000
104         feel  1518.000000  1518.000000  Default  27.0000  27.0000
258        great  1426.000000  1426.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
13          rule   156.553936  1059.886711  Topic14  -4.0162   0.9627
734        level    96.901780   206.138888  Topic14  -4.4959   2.1204
82     introduce    89.377136   128.499278  Topic14  -4.5767   2.5122
33         board   107.953146   822.783230  Topic14  -4.3879   0.8442
188  interesting   105.007095   902.051295  Topic14  -4.4156   0.7246

[578 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
111      12  1.000241  <number>
382       3  1.001526   ability
776       2  1.001483      able
32       11  0.998248  abstract
495       2  0.216793    action
...     ...       ...       ...
1680      2  0.997268     wrong
1505     12  0.998430         x
185       4  0.778539      year
185      11  0.222962      year
511       4  0.999256       yes

[630 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 5, 2, 4, 7, 8, 3, 13, 12, 6, 14, 10, 1, 9])

In [8]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(noun_lda, texts.apply(lambda x: noun_dictionary.doc2bow(x)).tolist(), noun_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.104087  0.162171       1        1  14.855029
8      0.043803  0.184094       2        1   9.697268
1      0.097994  0.189256       3        1   8.069661
4      0.050160 -0.142187       4        1   7.570154
5     -0.244340 -0.098283       5        1   6.943150
6     -0.136874 -0.012506       6        1   6.659817
0     -0.248575 -0.130831       7        1   6.474478
12     0.082840  0.138429       8        1   6.185736
10     0.076714  0.134510       9        1   6.039784
7      0.138706 -0.152739      10        1   5.834544
3     -0.189325 -0.026017      11        1   5.782674
13     0.188266 -0.267103      12        1   5.627898
11     0.033478  0.115684      13        1   5.452477
9      0.211238 -0.094478      14        1   4.807328, topic_info=           Term         Freq        Total Category  logprob  loglift
37         card  5176.000000  5176.000000  Default  30.0000  30.0000
10       player  5699.000000  5699.000000  Default  29.0000  29.0000
5           fun  2400.000000  2400.000000  Default  28.0000  28.0000
95         rule  2110.000000  2110.000000  Default  27.0000  27.0000
164    mechanic  1693.000000  1693.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
141   component   212.396172   860.054503  Topic14  -3.4993   1.6365
602       shelf    70.213900    98.457488  Topic14  -4.6062   2.6970
7         board   158.302217  1722.009593  Topic14  -3.7933   0.6483
314  production    90.972636   340.710317  Topic14  -4.3472   1.7146
72          way    65.859061  1455.957469  Topic14  -4.6703  -0.0609

[659 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
196       1  0.217199   ability
196       2  0.622636   ability
196       4  0.156383   ability
354      11  0.998358  abstract
458       1  0.462790    action
...     ...       ...       ...
331       8  0.838091     world
331      13  0.159842     world
1054     14  1.000753       wow
101       5  0.325016      year
101      13  0.674718      year

[766 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 2, 5, 6, 7, 1, 13, 11, 8, 4, 14, 12, 10])

In [ ]:
# Topic 1 identifies Downtime for example. Yet it also does crash with maybe with bookkeeping.
# We have to better tune the parameters to check if we can create better communities

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.